In [51]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from scipy.stats import ttest_ind, ttest_rel

In [52]:
customer_data = pd.read_csv("marketing_campaign.csv",sep="\t")
customer_data

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,43,182,42,118,247,2,9,3,4,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,0,30,0,0,8,7,8,2,5,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,48,217,32,12,24,1,2,3,13,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,30,214,80,30,61,2,6,5,10,3,0,0,0,0,0,0,3,11,0


In [53]:
for ed in customer_data['Education'].unique():
    print(ed, len(customer_data[customer_data.Education == ed]))

Graduation 1127
PhD 486
Master 370
Basic 54
2n Cycle 203


In [54]:
grad = ['PhD', 'Master']
df_ug = customer_data[customer_data.Education == 'Graduation'].copy()
df_ga = customer_data[customer_data.Education.isin(grad)].copy()
df_ug['Income'].fillna(df_ug['Income'].mean(), inplace=True)
df_ga['Income'].fillna(df_ga['Income'].mean(), inplace=True)
df_ug.iloc[:, 9:15] = df_ug.iloc[:, 9:15].fillna(df_ug.iloc[:, 9:15].mean())
df_ga.iloc[:, 9:15] = df_ga.iloc[:, 9:15].fillna(df_ga.iloc[:, 9:15].mean())
df_ug['Food'] = df_ug['MntFruits'] + df_ug['MntMeatProducts'] + df_ug['MntFishProducts']
df_ug['Other'] = df_ug['MntWines'] + df_ug['MntGoldProds'] + df_ug['MntSweetProducts']
df_ga['Food'] = df_ga['MntFruits'] + df_ga['MntMeatProducts'] + df_ga['MntFishProducts']
df_ga['Other'] = df_ga['MntWines'] + df_ga['MntGoldProds'] + df_ga['MntSweetProducts']

In [55]:
customer_data['Income'] = customer_data['Income'].fillna(customer_data['Income'].mean())
customer_data.iloc[:, 9:15] = customer_data.iloc[:, 9:15].fillna(customer_data.iloc[:, 9:15].mean())
customer_data['Food'] = customer_data['MntFruits'] + customer_data['MntMeatProducts'] + customer_data['MntFishProducts']
customer_data['Other'] = customer_data['MntWines'] + customer_data['MntGoldProds'] + customer_data['MntSweetProducts']

In [56]:
income_undergrad = df_ug['Income']
income_grad = df_ga['Income']

In [57]:
t, p = ttest_ind(income_undergrad, income_grad)
p

0.07324986756612731

In [58]:
food_ug = df_ug['Food'].to_numpy()
food_ga = df_ga['Food'].to_numpy()
t, p = ttest_ind(food_ug, food_ga)
p

0.004432271985434402

In [59]:
other_ug = df_ug['Other'].to_numpy()
other_ga = df_ga['Other'].to_numpy()
t, p = ttest_ind(other_ug, other_ga)
p

0.00025639375022679155

In [60]:
food = customer_data['Food'].to_numpy()
other = customer_data['Other'].to_numpy()
income = customer_data['Income'].to_numpy()
M = np.array([income, food, other])
M.shape

(3, 2240)

In [61]:
np.corrcoef(M)

array([[1.        , 0.59894015, 0.60402655],
       [0.59894015, 1.        , 0.63239864],
       [0.60402655, 0.63239864, 1.        ]])

In [62]:
df = pd.read_csv("marketing_campaign.csv",sep="\t")

In [63]:
df["age"] = 2014 - df["Year_Birth"]

In [64]:
df["Education"] = df["Education"].replace({"Basic":1,"2n Cycle": 2, "Graduation":3,
                         "Master":4,"PhD":5})

In [65]:
df = df.fillna(df['Income'].mean())
df["agg_food"] = df["MntFruits"] + df["MntMeatProducts"] + df["MntFishProducts"]
df["agg_other"] = df["MntWines"] + df["MntSweetProducts"] + df["MntGoldProds"]

In [66]:
df_temp = df.sample(frac=1).reset_index(drop=True)
feature = df_temp[["Education","Income","Kidhome","Teenhome","age"]]
label = df_temp[["agg_food","agg_other"]]
feature_train = feature.iloc[:1600]
feature_test = feature.iloc[1600:]
label_train = label.iloc[:1600]
label_test = label.iloc[1600:]
scaler= StandardScaler()
feature_train = scaler.fit_transform(feature_train)
feature_test = scaler.fit_transform(feature_test)
print(feature_train)

[[ 0.54907235  0.11133424 -0.81762174  0.91827561  1.41059112]
 [-0.44248043 -1.01479384  1.05658283 -0.9228785  -1.75179605]
 [-1.43403321  0.24900732 -0.81762174  0.91827561 -0.66992675]
 ...
 [-1.43403321  0.27012977 -0.81762174  0.91827561  0.32872182]
 [-2.425586   -1.06054657  1.05658283 -0.9228785  -0.75314747]
 [ 1.54062514 -0.75842016  1.05658283 -0.9228785  -0.17060247]]


In [67]:
poly = PolynomialFeatures(degree=2)
feature_train = poly.fit_transform(feature_train)
feature_test = poly.fit_transform(feature_test)

In [68]:
print(poly.get_feature_names(["Education","Income","Kidhome","Teenhome","age"]))

['1', 'Education', 'Income', 'Kidhome', 'Teenhome', 'age', 'Education^2', 'Education Income', 'Education Kidhome', 'Education Teenhome', 'Education age', 'Income^2', 'Income Kidhome', 'Income Teenhome', 'Income age', 'Kidhome^2', 'Kidhome Teenhome', 'Kidhome age', 'Teenhome^2', 'Teenhome age', 'age^2']


In [69]:
feature_train = pd.DataFrame(feature_train)
feature_test = pd.DataFrame(feature_test)
print(label_train)
label_train_1 = label_train.iloc[:,0]
label_train_2 = label_train.iloc[:,1]
label_test_1 = label_test.iloc[:,0]
label_test_2 = label_test.iloc[:,1]

      agg_food  agg_other
0           80       1294
1           19         23
2           76        721
3           36         52
4            7         48
...        ...        ...
1595       801        436
1596        16         43
1597       302        769
1598        43         37
1599        17         29

[1600 rows x 2 columns]


In [70]:

print(feature_train)
df_coef = pd.DataFrame(columns=[poly.get_feature_names(["Education","Income","Kidhome","Teenhome","age"])])
reg = Ridge(alpha=300).fit(feature_train, label_train_1)
print("The training set R^2 score, testing set R^2 score, and testing set MAE for food sector are shown below:")
print(reg.score(feature_train, label_train_1))
print(reg.score(feature_test, label_test_1))
print(mean_absolute_error(label_test_1,reg.predict(feature_test)))
df_coef = df_coef.append(pd.DataFrame(reg.coef_.reshape(1,-1), columns=list(df_coef), index=["food"]))

reg = Ridge(alpha=300).fit(feature_train, label_train_2)
print("The training set R^2 score, testing set R^2 score, and testing set MAE for other sector are shown below:")
print(reg.score(feature_train, label_train_2))
print(reg.score(feature_test, label_test_2))
print(mean_absolute_error(label_test_2,reg.predict(feature_test)))
df_coef = df_coef.append(pd.DataFrame(reg.coef_.reshape(1,-1), columns=list(df_coef), index=["other"]))

       0         1         2         3         4         5         6   \
0     1.0  0.549072  0.111334 -0.817622  0.918276  1.410591  0.301480   
1     1.0 -0.442480 -1.014794  1.056583 -0.922878 -1.751796  0.195789   
2     1.0 -1.434033  0.249007 -0.817622  0.918276 -0.669927  2.056451   
3     1.0 -2.425586 -1.431170  1.056583  0.918276  0.911267  5.883467   
4     1.0 -0.442480 -0.510948  1.056583 -0.922878 -1.169251  0.195789   
...   ...       ...       ...       ...       ...       ...       ...   
1595  1.0 -0.442480  1.176698 -0.817622  0.918276 -0.503485  0.195789   
1596  1.0  0.549072 -0.672195  1.056583 -0.922878 -0.170602  0.301480   
1597  1.0 -1.434033  0.270130 -0.817622  0.918276  0.328722  2.056451   
1598  1.0 -2.425586 -1.060547  1.056583 -0.922878 -0.753147  5.883467   
1599  1.0  1.540625 -0.758420  1.056583 -0.922878 -0.170602  2.373526   

            7         8         9         10        11        12        13  \
0     0.061131 -0.448933  0.504200  0.774517 

In [71]:
pd.set_option('max_columns', None)
print(df_coef)


         1  Education      Income     Kidhome   Teenhome       age  \
food   0.0 -13.327560  141.779490  -72.540013 -69.290691 -6.909167   
other  0.0  27.303132  203.073008 -102.916359 -20.332452  4.649499   

      Education^2 Education Income Education Kidhome Education Teenhome  \
food    -8.676481        -3.910464          1.070941          -1.015729   
other    3.758612       -12.760709        -28.218603          -2.867932   

      Education age  Income^2 Income Kidhome Income Teenhome Income age  \
food      -0.406269 -4.975157     -58.926733      -22.069669 -12.277175   
other     -3.290728 -8.781122     -21.843909       -7.843541   8.508549   

      Kidhome^2 Kidhome Teenhome Kidhome age Teenhome^2 Teenhome age     age^2  
food  -1.729205        33.631456    3.748476  19.856623    -4.015083  2.124565  
other  7.715261         2.911204    9.570800  10.471169     0.999681 -1.638216  


In [72]:
df_temp = df.sample(frac=1).reset_index(drop=True)
df_temp['diff_days'] = pd.to_datetime('20141231', format='%Y%m%d', errors='ignore') - pd.to_datetime(df['Dt_Customer'])
df_temp['diff_days']=df_temp['diff_days']/np.timedelta64(1,'D')

feature = df_temp.drop(['agg_food', 'agg_other', "MntFruits", 'MntMeatProducts',
                        "MntFishProducts","MntWines","MntGoldProds","ID","Year_Birth",
                        "Z_Revenue","Complain","Z_CostContact","Marital_Status","Dt_Customer","MntSweetProducts"], axis=1)
label = df_temp[["agg_food","agg_other"]]
dummy1 = pd.get_dummies(df_temp['Marital_Status'])
feature = pd.concat([feature, dummy1], axis=1)
print(feature.columns)

feature_train = feature.iloc[:1600]
feature_test = feature.iloc[1600:]
label_train = label.iloc[:1600]
label_test = label.iloc[1600:]
scaler= StandardScaler()
feature_train = scaler.fit_transform(feature_train)
feature_test = scaler.fit_transform(feature_test)

inputs = keras.Input(shape=(26,))
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)

x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
outputs = layers.Dense(2)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")

model.compile(
    loss="mae",
    optimizer="adam",
    metrics="mae",
)

Index(['Education', 'Income', 'Kidhome', 'Teenhome', 'Recency',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Response', 'age', 'diff_days', 'Absurd', 'Alone', 'Divorced',
       'Married', 'Single', 'Together', 'Widow', 'YOLO'],
      dtype='object')


In [76]:
history = model.fit(feature_train, label_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
20/20 [==============================] - 0s 2ms/step - loss: 113.2100 - mae: 113.2100 - val_loss: 113.8826 - val_mae: 113.8826
Epoch 2/20
20/20 [==============================] - 0s 1ms/step - loss: 112.8233 - mae: 112.8233 - val_loss: 112.7265 - val_mae: 112.7265
Epoch 3/20
20/20 [==============================] - 0s 1ms/step - loss: 112.4782 - mae: 112.4782 - val_loss: 112.3538 - val_mae: 112.3538
Epoch 4/20
20/20 [==============================] - 0s 1ms/step - loss: 111.4998 - mae: 111.4998 - val_loss: 111.9341 - val_mae: 111.9341
Epoch 5/20
20/20 [==============================] - 0s 1ms/step - loss: 110.7254 - mae: 110.7254 - val_loss: 111.9812 - val_mae: 111.9812
Epoch 6/20
20/20 [==============================] - 0s 1ms/step - loss: 110.2967 - mae: 110.2967 - val_loss: 111.0763 - val_mae: 111.0

In [77]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
print("The r^2 scores and MAE for the testing set is : ")
print(r2_score(label_test,model.predict(feature_test)))
print(mean_absolute_error(label_test,model.predict(feature_test)))

The r^2 scores and MAE for the testing set is : 
0.7383104683578654
100.70975775644183


In [75]:
a = np.mean(label_train)
sum = 0
for i in label.to_numpy():
    t = i - a
    for j in t:
        if (j > 0):
            sum = sum + j
        else:
            sum = sum - j
print(sum/(2*len(label_train)))
print(np.std(label))



379.477062500005
agg_food     286.060806
agg_other    378.950805
dtype: float64
